In [8]:
pip install hazm

Note: you may need to restart the kernel to use updated packages.


In [9]:
from hazm import *


In [11]:
tagger = POSTagger(model='pos_tagger.model')
tagger.tag(word_tokenize('ما بسیار کتاب می‌خوانیم'))
[('ما', 'PRO'), ('بسیار', 'ADV'), ('کتاب', 'N'), ('می‌خوانیم', 'V')]


[('ما', 'PRO'), ('بسیار', 'ADV'), ('کتاب', 'N'), ('می\u200cخوانیم', 'V')]

In [22]:
import pandas as pd
from hazm import Normalizer, word_tokenize, POSTagger

# edit paths as needed
IN_CSV  = "data/test.csv"
OUT_CSV = "data/pairs_pos_sample.csv"
MODEL   = "pos_tagger.model"           # e.g. "resources/postagger.model"
DELIM   = ","                          # change if your CSV uses another delimiter

# read only first 10 rows
df = pd.read_csv(IN_CSV, delimiter=DELIM, nrows=10)

tagger = POSTagger(model=MODEL)
norm = Normalizer(persian_numbers=True)

def pos_tag_line(text: str) -> str:
    if not isinstance(text, str) or not text.strip():
        return ""
    tokens = word_tokenize(norm.normalize(text))
    return " ".join(f"{tok}/{pos}" for tok, pos in tagger.tag(tokens))

df["persian_pos"] = df["persian"].apply(pos_tag_line)

df.to_csv(OUT_CSV, index=False)
print("Saved ->", OUT_CSV)
# print(df[["persian", "persian_pos"]].head(10))


Saved -> data/pairs_pos_sample.csv
                                             persian  \
0  رئیس‌جمهور تاکید کرد: کار این دولت فقط با نتیج...   
1                          من دیگر تو را دوست ندارم.   
2                   ما یک خانه کامل در بلوک b داریم.   
3  عکس‌های خود را به pr@cuibul. com بفرستید یا در...   
4                 نه. این فقط شما را عالی‌تر می‌کند.   
5                      1396 / 12 / 9 12: 05: 56 ق. ظ   
6                                      (39 کیلوبایت)   
7               Herscher (Illinois) 815426 **** تلفن   
8  همه‌ی چیزایی که «چندلر» از روی. احساس گناه گرف...   
9                  11. موتور تون جایزه بزرگ (مسابقه)   

                                         persian_pos  
0  رئیس‌جمهور/NOUN تاکید/NOUN کرد/VERB :/PUNCT کا...  
1  من/PRON دیگر/ADV تو/PRON را/ADP دوست/NOUN ندار...  
2  ما/PRON یک/NUM خانه/NOUN,EZ کامل/ADJ در/ADP بل...  
3  عکس‌های/NOUN,EZ خود/PRON را/ADP به/ADP pr@cuib...  
4  نه/ADV ./PUNCT این/PRON فقط/ADV شما/PRON را/AD...  
5  ۱۳۹۶/NUM //PUNC

In [23]:
import pandas as pd

df = pd.read_csv("data/pairs_pos_sample.csv")  # مسیر را اگر لازم است عوض کن

def parse_toktag(s: str):
    toks, tags = [], []
    if isinstance(s, str):
        for chunk in s.split():
            if "/" in chunk:
                tok, tag = chunk.rsplit("/", 1)
            else:
                tok, tag = chunk, ""
            toks.append(tok); tags.append(tag)
    return toks, tags

def pretty_block(s: str):
    toks, tags = parse_toktag(s)
    if not toks:
        return ""
    w = max(len(t) for t in toks)  # عرض ستون توکن
    # هر خط:  token(left padded)   tag
    lines = [f"{t:<{w}}   {g}" for t, g in zip(toks, tags)]
    return "\n".join(lines)

pd.set_option("display.max_colwidth", None)  # قطع نکردن متن‌های بلند
df["pos_pretty"] = df["persian_pos"].map(pretty_block)

# نمایش شکیل (حفظ \n داخل سلول + فونت مونو)
df[["persian", "pos_pretty"]].style.set_properties(
    subset=["pos_pretty"],
    **{"white-space": "pre", "font-family": "monospace"}
)


,persian,pos_pretty
0,رئیس‌جمهور تاکید کرد: کار این دولت فقط با نتیجه ارزیابی می‌شود.,"رئیس‌جمهور NOUN تاکید NOUN کرد VERB : PUNCT کار NOUN,EZ این DET دولت NOUN فقط ADV با ADP نتیجه NOUN ارزیابی NOUN می‌شود VERB . PUNCT"
1,من دیگر تو را دوست ندارم.,من PRON دیگر ADV تو PRON را ADP دوست NOUN ندارم VERB . PUNCT
2,ما یک خانه کامل در بلوک b داریم.,"ما PRON یک NUM خانه NOUN,EZ کامل ADJ در ADP بلوک NOUN,EZ b NOUN داریم VERB . PUNCT"
3,عکس‌های خود را به pr@cuibul. com بفرستید یا در جایی آنلاین قرار دهید و لینک را به همان آدرس ارسال کنید. فراموش نکنید که نام خود را ذکر کنید و شاید یک …,"عکس‌های NOUN,EZ خود PRON را ADP به ADP pr@cuibul NOUN . PUNCT com NOUN بفرستید VERB یا CCONJ در ADP جایی NOUN آنلاین NOUN قرار NOUN دهید VERB و CCONJ لینک NOUN را ADP به ADP همان DET آدرس NOUN ارسال NOUN کنید VERB . PUNCT فراموش ADJ نکنید VERB که SCONJ نام NOUN,EZ خود PRON را ADP ذکر NOUN کنید VERB و CCONJ شاید ADV یک NUM … PUNCT"
4,نه. این فقط شما را عالی‌تر می‌کند.,نه ADV . PUNCT این PRON فقط ADV شما PRON را ADP عالی‌تر ADJ می‌کند VERB . PUNCT
5,1396 / 12 / 9 12: 05: 56 ق. ظ,"۱۳۹۶ NUM / PUNCT ۱۲ NUM / PUNCT ۹ NUM ۱۲: NUM,EZ ۰۵: NOUN,EZ ۵۶ NUM ق NOUN . PUNCT ظ NOUN"
6,(39 کیلوبایت),( PUNCT ۳۹ NUM کیلوبایت NOUN ) PUNCT
7,Herscher (Illinois) 815426 **** تلفن,Herscher NOUN ( PUNCT Illinois NOUN ) PUNCT ۸۱۵۴۲۶ NUM **** NOUN تلفن NOUN
8,همه‌ی چیزایی که «چندلر» از روی. احساس گناه گرفته رو پس میدم,"همه‌ی DET,EZ چیزایی NOUN که SCONJ « PUNCT چندلر NOUN » PUNCT از ADP روی NOUN . PUNCT احساس NOUN,EZ گناه NOUN گرفته VERB رو ADP پس ADP میدم VERB"
9,11. موتور تون جایزه بزرگ (مسابقه),"۱۱. NOUN موتور NOUN,EZ تون NOUN جایزه NOUN,EZ بزرگ ADJ ( PUNCT مسابقه NOUN ) PUNCT"
